In [20]:
import numpy as np
import pandas as pd

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split

In [21]:
df = pd.read_csv('covid_toy.csv')

In [22]:
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [23]:
df['city'].value_counts()

Kolkata      32
Bangalore    30
Delhi        22
Mumbai       16
Name: city, dtype: int64

In [24]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [10]:
#df = df.fillna('')

In [25]:
#df.isnull().sum()

In [26]:
x_train, x_test, y_train, y_test = train_test_split(df.drop(columns =['has_covid']), df['has_covid'], test_size = 0.2)

In [27]:
x_train

,age,gender,fever,cough,city
86,25,Male,104.0,Mild,Bangalore
12,25,Female,99.0,Strong,Kolkata
28,16,Male,104.0,Mild,Kolkata
7,20,Female,NaN,Strong,Mumbai
75,5,Male,102.0,Mild,Kolkata
...,...,...,...,...,...
39,50,Female,103.0,Mild,Kolkata
48,66,Male,99.0,Strong,Bangalore
25,23,Male,NaN,Mild,Mumbai
44,20,Male,102.0,Strong,Delhi


In [28]:
## Processing without column Transform function

In [30]:
#adding single imputer to fewer col

sl =SimpleImputer()
x_train_fever = sl.fit_transform(x_train[['fever']])

#also the test data
x_test_fever = sl.fit_transform(x_test[['fever']])

In [33]:
x_train_fever.shape

(80, 1)

In [36]:
# OrdinalEncoding -> cough
oe = OrdinalEncoder(categories = [['Mild','Strong']])
x_train_cough = oe.fit_transform(x_train[['cough']])

#also the test data 
x_test_cough = oe.fit_transform(x_test[['cough']])

x_train_cough.shape

(80, 1)

In [38]:
#Onehotencoding -> gender , city
ohe = OneHotEncoder(drop = 'first', sparse = False)
x_train_gender_city = ohe.fit_transform(x_train[['gender','city']])

#also the test data
x_test_gender_city = ohe.fit_transform(x_test[['gender','city']])

x_train_gender_city.shape


(80, 4)

In [39]:
# Extracting  Age
x_train_age = x_train.drop(columns = ['gender','fever','cough','city']).values

#also the test data
x_test_age = x_test.drop(columns = ['gender','fever','cough','city']).values

x_train_age.shape

(80, 1)

In [41]:
# train data
x_train_transformed = np.concatenate((x_train_age, x_train_fever ,x_train_gender_city , x_train_cough), axis =1)
# test data
x_test_transformed = np.concatenate((x_test_age, x_test_fever ,x_test_gender_city , x_test_cough), axis =1)

x_train_transformed.shape

(80, 7)

In [42]:
## Using Colum Transform from sklearn

from sklearn.compose import ColumnTransformer

In [46]:
transformer = ColumnTransformer(transformers =[
    ('tnf1', SimpleImputer(),['fever']),
    ('tnf2', OrdinalEncoder(categories =[['Mild','Strong']]),['cough']),
    ('tnf3', OneHotEncoder(sparse = False , drop ='first'),['gender','city'])
], remainder = 'passthrough')

In [47]:
transformer.fit_transform(x_train).shape

(80, 7)

In [48]:
transformer.transform(x_test)

array([[ 98.        ,   1.        ,   0.        ,   1.        ,
          0.        ,   0.        ,  40.        ],
       [ 98.        ,   1.        ,   0.        ,   0.        ,
          0.        ,   1.        ,  81.        ],
       [101.        ,   1.        ,   0.        ,   1.        ,
          0.        ,   0.        ,  34.        ],
       [100.        ,   0.        ,   1.        ,   1.        ,
          0.        ,   0.        ,  27.        ],
       [ 99.        ,   0.        ,   1.        ,   1.        ,
          0.        ,   0.        ,  65.        ],
       [103.        ,   1.        ,   1.        ,   0.        ,
          0.        ,   0.        ,  46.        ],
       [ 98.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   1.        ,  65.        ],
       [ 98.        ,   0.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  26.        ],
       [100.        ,   0.        ,   1.        ,   0.        ,
          0.    